<br>

### Institut de Mathématiques et de Sciences Physiques de Dangbo Bénin
# MINI PROJET DE PROGRAMMATION EN NOMBRES ENTIERS
### Sujet: Résoudre par la méthode de Branch and Bound un problème simple de localisation d'entrepôts
#### Réalisé par : Larissa DENAKPO
#### Formation: Master 1 de Recherche Opérationnelle

# Problème:

Un pays souhaite construire des entrepôts pour satisfaire les demandes de son peuple:
- La capitale, Voldemort (qui a besoin de 500 unités de nourriture)
- La citadelle de Dalaran (besoin : 350 unités)
- La cité de Stratholme (besoin : 300 unités)
- La forteresse de Mardenhol (besoin : 400 unités)
- La forteresse de Durnholde (besoin : 350 unités)
- La cité de Caer (besoin : 250 unités)
 
(1) Les entrepôts peuvent être placés à Brill, Hearthglen, Andorhal et Strahnbrad

(2) Les coûts de constructions associés sont respectivement de 200, 250, 300 et 350 pièces d'or.

(3) Les coûts de transports entre les lieux possibles pour les entrepôts et les cités à desservir sont les suivants (en pièces d'or).

$\phantom{1111111111}$ Brill $\phantom{111}$ Hearthglen $\phantom{111}$ Andorhal $\phantom{111}$ Strahnbrad

Voldemort $\phantom{11}$ 100 $\phantom{11111}$ 200 $\phantom{11111111}$ 250  $\phantom{1111111}$ 350

Dalaran $\phantom{1111}$ 500 $\phantom{11111}$ 450 $\phantom{11111111}$ 350 $\phantom{1111111}$ 250

Stratholme$\phantom{11}$ 125 $\phantom{111111}$ 75 $\phantom{11111111}$ 125 $\phantom{1111111}$ 300

Mardenhol$\phantom{11}$ 175 $\phantom{111111}$ 50 $\phantom{11111111}$ 175 $\phantom{1111111}$ 300

Durnholde$\phantom{11}$ 300 $\phantom{111111}$ 300 $\phantom{1111111}$ 200 $\phantom{1111111}$ 100

Caer $\phantom{111111}$ 350 $\phantom{111111}$ 350 $\phantom{1111111}$ 250 $\phantom{1111111}$ 150

(4) La capacité des entrepôts est de chacun 800.

Typiquement, le problème est de placer les entrepôts, de manière à minimiser les coûts de constructions et de transport, tout en satisfaisant les différentes coontraintes (demandes, capacité).



# Modélisation

Soit $x_{ij}$ valant 1 si la ville $i$ est affectée à l'entrepôt au point $j$, 0 sinon.

Soit $y_j$ valant 1 si un entrepôt est construit au point $j$, 0 sinon.

Soit $c_{ij}$ le coût de transport de l'entrepôt au point $j$ à la ville $i$.

Soit $f_j$ la coût d'installation d'un entrepôt au point $j$.

Soit $b_i$ les besoins de chaque ville $i$.

Soit $B$ la quantité disponible pour chaque entrepôt.

$i \in I=\{1,...,6\}, \ j \in J=\{1,...,4\}$

L'objectif est de minimiser le coût total en tenant compte de satisfaire les demandes de chaque villes, des capacités des entrepôts et du fait qu'une ville ne peut être affectée à un entrepôt qui n'est pas construit.

\begin{align}
\text{minimize} \qquad & \sum_{j \in\ J}{f_j y_j} + \sum_{i\in\ I}\sum_{j\in\ J}{c_{ij}x_{ij}} \\
 \text{subject to} \quad & \sum_{j\in\ J}x_{ij} = 1,\ \forall \ i \in I\\
 \qquad \qquad & \sum_{i\in \ I}b_i x_{ij} \leq B, \ \forall\ j\in J\\
 \qquad \qquad & x_{ij} \leq y_j,\ \forall \ i\in I,\ j\in J \\
 \qquad \\
 \qquad \qquad & x_{ij} \in \{0,1\}, \ \forall \ i\in I, j\in J\\
 \qquad \qquad & y_j \in \{0,1\},\ \forall\ j\in J
\end{align}


In [2]:
using JuMP
using Cbc

In [3]:
#data
n = 6
m = 4

f = [200; 250; 300; 350]

c = [100 200 250 350;
     500 450 350 250;
     125 75 125 300;
     175 50 175 300;
     300 300 200 100;
     350 350 250 150
    ]

b = [500; 350; 300; 400; 350; 250]
B = 800;

Résolvons la relaxation linéaire du problème pour trouver une borne inférieure à la solution

In [5]:
#Relaxation linéaire

FacilitiesMod = Model(solver = CbcSolver());

@defVar(FacilitiesMod, 0 <= x[ 1:n, 1:m ] <= 1 )
@defVar(FacilitiesMod, 0 <= y[ 1:m ] <= 1)

@setObjective(FacilitiesMod, Min, sum{f[j]*y[j],j = 1:m} + sum{sum{c[i,j]*x[i,j],i = 1:n}, j = 1:m})

for i = 1:n
    @addConstraint(FacilitiesMod, sum{x[i,j], j = 1:m } == 1)
end

for j = 1:m
    @addConstraint(FacilitiesMod, sum{b[i]*x[i,j], i = 1:n } <= B)
end

for j = 1:m
    for i = 1:n
        @addConstraint(FacilitiesMod, x[i,j] <= y[j])
    end
end

solve(FacilitiesMod)

for j = 1:m
    print("y[", j, "] = ", getValue(y[j]),"; ")
end
println()
for i = 1:n
    for j = 1:m
        print("x[", i, "," ,j, "] = ", getValue(x[i,j]),"; ")
    end
    println()
end


println("\nObjectif = ", getObjectiveValue(FacilitiesMod))

y[1] = 0.17543859649122795; y[2] = 0.6666666666666667; y[3] = 0.1578947368421053; y[4] = 0.8421052631578947; 
x[1,1] = 0.17543859649122795; x[1,2] = 0.6666666666666667; x[1,3] = 0.1578947368421053; x[1,4] = 0.0; 
x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.1578947368421053; x[2,4] = 0.8421052631578947; 
x[3,1] = 0.17543859649122795; x[3,2] = 0.6666666666666667; x[3,3] = 0.1578947368421053; x[3,4] = 0.0; 
x[4,1] = 0.17543859649122795; x[4,2] = 0.6666666666666667; x[4,3] = 0.1578947368421053; x[4,4] = 0.0; 
x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.1578947368421053; x[5,4] = 0.8421052631578947; 
x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.1578947368421053; x[6,4] = 0.8421052631578947; 

Objectif = 1464.9122807017545


## Branchements
Le but ici est de trouver une solution réalisable en faisant un parcours en profondeur. On branchera chaque fois sur la variable la plus proche de 0,5 obtenue par relaxation linéaire. On notera chaque fois $Objectif = Infini$ si les contraintes ne sont pas respectées.

Commençons par ajouter la contrainte y[2] = 0

$y[2] = 0$

    y[1] = 0.6666666666666666; y[2] = 0.0; y[3] = 0.33333333333333326; y[4] = 0.6666666666666667; 
    x[1,1] = 0.6666666666666666; x[1,2] = 0.0; x[1,3] = 0.33333333333333326; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.33333333333333326; x[2,4] = 0.6666666666666667; 
    x[3,1] = 0.6666666666666667; x[3,2] = 0.0; x[3,3] = 0.33333333333333326; x[3,4] = 0.0; 
    x[4,1] = 0.6666666666666667; x[4,2] = 0.0; x[4,3] = 0.33333333333333326; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.33333333333333326; x[5,4] = 0.6666666666666667; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.33333333333333326; x[6,4] = 0.6666666666666667; 

$Objectif = 1516.6666666666665$

$y[2] = 1$

    y[1] = 0.34210526315789463; y[2] = 1.0; y[3] = 0.1578947368421053; y[4] = 0.8421052631578947; 
    x[1,1] = 0.3421052631578947; x[1,2] = 0.5; x[1,3] = 0.1578947368421053; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.1578947368421053; x[2,4] = 0.8421052631578947; 
    x[3,1] = 0.34210526315789463; x[3,2] = 0.5; x[3,3] = 0.1578947368421053; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.1578947368421053; x[5,4] = 0.8421052631578947; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.1578947368421053; x[6,4] = 0.8421052631578947; 
    
$Objectif = 1531.578947368421$

$y[2] = 0; x[1,1] = 0$

    y[1] = 0.7857142857142858; y[2] = 0.0; y[3] = 0.44827586206896564; y[4] = 0.5517241379310345;
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 0.44827586206896564; x[1,4] = 0.5517241379310344; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.4482758620689655; x[2,4] = 0.5517241379310345; 
    x[3,1] = 0.7857142857142858; x[3,2] = 0.0; x[3,3] = 0.2142857142857142; x[3,4] = 0.0; 
    x[4,1] = 0.7857142857142858; x[4,2] = 0.0; x[4,3] = 0.21428571428571414; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.4482758620689655; x[5,4] = 0.5517241379310345; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.4482758620689655; x[6,4] = 0.5517241379310345;  

$Objectif = 1724.384236453202$

$y[2] = 0; x[1,1] = 1$
    
    y[1] = 1.0; y[2] = 0.0; y[3] = 0.3333333333333333; y[4] = 0.6666666666666666; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.33333333333333337; x[2,4] = 0.6666666666666666; 
    x[3,1] = 0.6666666666666667; x[3,2] = 0.0; x[3,3] = 0.3333333333333333; x[3,4] = 0.0; 
    x[4,1] = 0.25; x[4,2] = 0.0; x[4,3] = 0.3333333333333333; x[4,4] = 0.4166666666666667; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.33333333333333337; x[5,4] = 0.6666666666666666; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.3333333333333333; x[6,4] = 0.6666666666666665; 
    
$Objectif = 1585.4166666666667$

$y[2] = 0; x[1,1] = 0; y[4] = 0$

$Objectif = Infini$

$y[2] = 0; x[1,1] = 0; y[4] = 1$

    y[1] = 0.5789473684210527; y[2] = 0.0; y[3] = 0.42105263157894735; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 0.42105263157894735; x[1,4] = 0.5789473684210527; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.42105263157894735; x[2,4] = 0.5789473684210527; 
    x[3,1] = 0.5789473684210527; x[3,2] = 0.0; x[3,3] = 0.42105263157894735; x[3,4] = 0.0; 
    x[4,1] = 0.5789473684210527; x[4,2] = 0.0; x[4,3] = 0.4210526315789474; x[4,4] = 0.0; 
    x[5,1] = 0.4135338345864663; x[5,2] = 0.0; x[5,3] = 0.42105263157894735; x[5,4] = 0.16541353383458632; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 1866.9172932330828$

$y[2] = 0; x[1,1] = 0; y[4] = 1; y[1] = 0$

$Objectif = Infini$

$y[2] = 0; x[1,1] = 0; y[4] = 1; y[1] = 1$

    y[1] = 1.0; y[2] = 0.0; y[3] = 0.5416666666666667; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 0.5416666666666667; x[1,4] = 0.45833333333333326; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.5416666666666667; x[2,4] = 0.45833333333333326; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 1.0; x[4,2] = 0.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.5416666666666667; x[5,4] = 0.45833333333333326; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 1916.6666666666667$

$y[2] = 0; x[1,1] = 0; y[4] = 1; y[1] = 1; y[3] = 0$

$Objectif = Infini$

$y[2] = 0; x[1,1] = 0; y[4] = 1; y[1] = 1; y[3] = 1$

    y[1] = 1.0; y[2] = 0.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 1.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 1.0; x[4,2] = 0.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.4285714285714286; x[5,4] = 0.5714285714285714; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 1942.857142857143$

$y[2] = 0; x[1,1] = 0; y[4] = 1; y[1] = 1; y[3] = 1; x[5,4] = 0$

    y[1] = 1.0; y[2] = 0.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 0.9; x[1,4] = 0.09999999999999998; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 1.0; x[4,2] = 0.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = -0.0; x[5,2] = 0.0; x[5,3] = 1.0; x[5,4] = 0.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 2010.0$

$y[2] = 0; x[1,1] = 0; y[4] = 1; y[1] = 1; y[3] = 1; x[5,4] = 1$

    y[1] = 1.0; y[2] = 0.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 1.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.4285714285714286; x[2,4] = 0.5714285714285714; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 1.0; x[4,2] = 0.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 1942.857142857143$

$y[2] = 0; x[1,1] = 0; y[4] = 1; y[1] = 1; y[3] = 1; x[5,4] = 0; x[1,3] = 0$

    y[1] = 1.0; y[2] = 0.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 1.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.8571428571428572; x[2,4] = 0.14285714285714285; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 1.0; x[4,2] = 0.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = -0.0; x[5,2] = 0.0; x[5,3] = 1.0; x[5,4] = 0.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 2185.714285714286$

$y[2] = 0; x[1,1] = 0; y[4] = 1; y[1] = 1; y[3] = 1; x[5,4] = 0; x[1,3] = 1$

    y[1] = 1.0; y[2] = 0.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 1.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 1.0; x[4,2] = 0.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.14285714285714285; x[5,2] = 0.0; x[5,3] = 0.8571428571428572; x[5,4] = 0.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 2014.2857142857142$

$y[2] = 0; x[1,1] = 0; y[4] = 1; y[1] = 1; y[3] = 1; x[5,4] = 0; x[1,3] = 0; x[2,3] = 0$

    y[1] = 1.0; y[2] = 0.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 1.0; 
    x[2,1] = 0.14285714285714285; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 0.8571428571428572; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 1.0; x[4,2] = 0.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = -0.0; x[5,2] = 0.0; x[5,3] = 1.0; x[5,4] = 0.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 1.0; x[6,4] = 0.0; 

$Objectif = 2235.714285714286$

$y[2] = 0; x[1,1] = 0; y[4] = 1; y[1] = 1; y[3] = 1; x[5,4] = 0; x[1,3] = 0; x[2,3] = 1$

    y[1] = 1.0; y[2] = 0.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 1.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 1.0; x[2,4] = 0.0; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 1.0; x[4,2] = 0.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = -0.0; x[5,2] = 0.0; x[5,3] = 1.0; x[5,4] = 0.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 2200.0$

### Branchements + Elagage
On obtient à ce stade une solution réalisable. On a donc comme bornes de notre solution optimale la valeur de la relaxation linéaire et celle de la solution réalisable:
$$]1464.9122807017545,_ \ 2200]$$
Toute valeur de l'objectif n'appartenant pas à cet intervalle sera élaguée.

Par exemple; pour le branchement précédent: $y[2] = 0; x[1,1] = 0; y[4] = 1; y[1] = 1; y[3] = 1; x[5,4] = 0; x[1,3] = 0; x[2,3] = 0$, la valeur de l'objectif obtenue étant

$Objectif = 2235.714285714286 > 2200$, on élage ce sous arbre.

$y[2] = 0; x[1,1] = 0; y[4] = 1; y[1] = 1; y[3] = 1; x[5,4] = 0; x[1,3] = 1; x[5,3] = 0$

    y[1] = 1.0; y[2] = 0.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 1.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.375; x[4,2] = 0.0; x[4,3] = 0.625; x[4,4] = 0.0; 
    x[5,1] = 1.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 0.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 2100.0$

$y[2] = 0; x[1,1] = 0; y[4] = 1; y[1] = 1; y[3] = 1; x[5,4] = 0; x[1,3] = 1; x[5,3] = 1$

$Objectif = Infini$

$y[2] = 0; x[1,1] = 0; y[4] = 1; y[1] = 1; y[3] = 1; x[5,4] = 0; x[1,3] = 1; x[5,3] = 0; x[4,3] = 0$

    y[1] = 1.0; y[2] = 0.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 1.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 0.16666666666666666; x[3,2] = 0.0; x[3,3] = 0.8333333333333334; x[3,4] = 0.0; 
    x[4,1] = 1.0; x[4,2] = 0.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 1.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 0.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 2100.0$

$y[2] = 0; x[1,1] = 0; y[4] = 1; y[1] = 1; y[3] = 1; x[5,4] = 0; x[1,3] = 1; x[5,3] = 0; x[4,3] = 1$

$Objectif = Infini$

$y[2] = 0; x[1,1] = 0; y[4] = 1; y[1] = 1; y[3] = 1; x[5,4] = 0; x[1,3] = 1; x[5,3] = 0; x[4,3] = 0; x[3,1] = 0$

    y[1] = 1.0; y[2] = 0.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 1.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 0.0; x[3,2] = 0.0; x[3,3] = 1.0; x[3,4] = 0.0; 
    x[4,1] = 1.0; x[4,2] = 0.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 1.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 0.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 2100.0$

La solution obtenue étant réalisable et $2100 < 2200$; l'intervalle devient: $$]1464.9122807017545,_ \ 2100]$$

$y[2] = 0; x[1,1] = 0; y[4] = 1; y[1] = 1; y[3] = 1; x[5,4] = 0; x[1,3] = 1; x[5,3] = 0; x[4,3] = 0; x[3,1] = 1$

    y[1] = 1.0; y[2] = 0.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 1.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.1428571428571429; x[2,4] = 0.8571428571428571; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.375; x[4,2] = 0.0; x[4,3] = 0.0; x[4,4] = 0.625; 
    x[5,1] = 1.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 0.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 2192.410714285714 > 2100$, on élague

$y[2] = 0; x[1,1] = 0; y[4] = 1; y[1] = 1; y[3] = 1; x[5,4] = 1, x[2,4] = 0$

    y[1] = 1.0; y[2] = 0.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 0.9; x[1,4] = 0.09999999999999998; 
    x[2,1] = -0.0; x[2,2] = 0.0; x[2,3] = 1.0; x[2,4] = 0.0; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 1.0; x[4,2] = 0.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 2010.0$

$y[2] = 0; x[1,1] = 0; y[4] = 1; y[1] = 1; y[3] = 1; x[5,4] = 1, x[2,4] = 1$

    y[1] = 1.0; y[2] = 0.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 1.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 1.0; x[4,2] = 0.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.6000000000000001; x[6,4] = 0.3999999999999999; 

$Objectif = 1960.0$

$y[2] = 0; x[1,1] = 0; y[4] = 1; y[1] = 1; y[3] = 1; x[5,4] = 1, x[2,4] = 0, x[1,3] = 0$

$Objectif = Infini$

$y[2] = 0; x[1,1] = 0; y[4] = 1; y[1] = 1; y[3] = 1; x[5,4] = 1, x[2,4] = 0, x[1,3] = 1$

    y[1] = 1.0; y[2] = 0.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 1.0; x[1,4] = 0.0; 
    x[2,1] = 0.14285714285714285; x[2,2] = 0.0; x[2,3] = 0.8571428571428572; x[2,4] = 0.0; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 1.0; x[4,2] = 0.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 2021.4285714285716$

$y[2] = 0; x[1,1] = 0; y[4] = 1; y[1] = 1; y[3] = 1; x[5,4] = 1, x[2,4] = 0, x[1,3] = 1, x[2,3] = 0$

    y[1] = 1.0; y[2] = 0.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 1.0; x[1,4] = 0.0; 
    x[2,1] = 1.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 0.0; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.375; x[4,2] = 0.0; x[4,3] = 0.625; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 2150.0 > 2100$, on élague

$y[2] = 0; x[1,1] = 0; y[4] = 1; y[1] = 1; y[3] = 1; x[5,4] = 1, x[2,4] = 0, x[1,3] = 1, x[2,3] = 1$

$Objectif = Infini$

$y[2] = 0; x[1,1] = 0; y[4] = 1; y[1] = 1; y[3] = 1; x[5,4] = 1, x[2,4] = 1, x[6,3] = 0$

    y[1] = 1.0; y[2] = 0.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 1.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.875; x[4,2] = 0.0; x[4,3] = 0.125; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.6000000000000001; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 0.3999999999999999; 

$Objectif = 2020.0$

$y[2] = 0; x[1,1] = 0; y[4] = 1; y[1] = 1; y[3] = 1; x[5,4] = 1, x[2,4] = 1, x[6,3] = 1$

    y[1] = 1.0; y[2] = 0.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 1.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 1.0; x[4,2] = 0.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 1.0; x[6,4] = 0.0; 

$Objectif = 2000.0$

L'intervalle devient: $$]1464.9122807017545,_ \ 2000]$$

On élague donc le sous arbre $y[2] = 0; x[1,1] = 0; y[4] = 1; y[1] = 1; y[3] = 1; x[5,4] = 1, x[2,4] = 1, x[6,3] = 0$ car :

$Objectif = 2020 > 2000$

$y[2] = 0; x[1,1] = 1, x[4,4] = 0$

    y[1] = 1.0; y[2] = 0.0; y[3] = 0.5714285714285714; y[4] = 0.5789473684210527; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.42105263157894735; x[2,4] = 0.5789473684210527; 
    x[3,1] = 0.4285714285714286; x[3,2] = 0.0; x[3,3] = 0.5714285714285714; x[3,4] = 0.0; 
    x[4,1] = 0.42857142857142855; x[4,2] = 0.0; x[4,3] = 0.5714285714285714; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.42105263157894735; x[5,4] = 0.5789473684210527; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.42105263157894735; x[6,4] = 0.5789473684210527; 

$Objectif = 1600.375939849624$

$y[2] = 0; x[1,1] = 1, x[4,4] = 1$

    y[1] = 1.0; y[2] = 0.0; y[3] = 0.44000000000000006; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.44; x[2,4] = 0.5600000000000002; 
    x[3,1] = 0.5599999999999999; x[3,2] = 0.0; x[3,3] = 0.44000000000000006; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 0.0; x[4,3] = 0.0; x[4,4] = 1.0; 
    x[5,1] = 0.37714285714285717; x[5,2] = 0.0; x[5,3] = 0.44000000000000006; x[5,4] = 0.18285714285714272; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.44000000000000006; x[6,4] = 0.5599999999999999; 

$Objectif = 1914.4285714285713$

$y[2] = 0; x[1,1] = 1, x[4,4] = 0; y[4] = 0$

$Objectif = Infini$

$y[2] = 0; x[1,1] = 1, x[4,4] = 0; y[4] = 1$

    y[1] = 1.0; y[2] = 0.0; y[3] = 0.5714285714285714; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 0.42857142857142855; x[3,2] = 0.0; x[3,3] = 0.5714285714285714; x[3,4] = 0.0; 
    x[4,1] = 0.42857142857142855; x[4,2] = 0.0; x[4,3] = 0.5714285714285714; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.4285714285714288; x[5,4] = 0.5714285714285712; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 1664.2857142857142$

$y[2] = 0; x[1,1] = 1, x[4,4] = 0; y[4] = 1; y[3] = 0$

$Objectif = Infini$

$y[2] = 0; x[1,1] = 1, x[4,4] = 0; y[4] = 1; y[3] = 1$

    y[1] = 1.0; y[2] = 0.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.4285714285714288; x[2,4] = 0.5714285714285712; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = -0.0; x[4,2] = 0.0; x[4,3] = 1.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 1792.857142857143$

$y[2] = 0; x[1,1] = 1, x[4,4] = 0; y[4] = 1; y[3] = 1; x[2,4] = 0$

    y[1] = 1.0; y[2] = 0.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = -0.0; x[2,2] = 0.0; x[2,3] = 1.0; x[2,4] = 0.0; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = -0.0; x[4,2] = 0.0; x[4,3] = 1.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 1850.0$

L'intervalle devient: $$]1464.9122807017545,_ \ 1850]$$

$y[2] = 0; x[1,1] = 1, x[4,4] = 0; y[4] = 1; y[3] = 1; x[2,4] = 1$

    y[1] = 1.0; y[2] = 0.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = -0.0; x[4,2] = 0.0; x[4,3] = 1.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.4285714285714285; x[5,4] = 0.5714285714285715; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 1792.8571428571427$


$y[2] = 0; x[1,1] = 1, x[4,4] = 0; y[4] = 1; y[3] = 1; x[2,4] = 1; x[5,4] = 0$

    y[1] = 1.0; y[2] = 0.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = -0.0; x[4,2] = 0.0; x[4,3] = 1.0; x[4,4] = 0.0; 
    x[5,1] = -0.0; x[5,2] = 0.0; x[5,3] = 1.0; x[5,4] = 0.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 1850.0$

$y[2] = 0; x[1,1] = 1, x[4,4] = 0; y[4] = 1; y[3] = 1; x[2,4] = 1; x[5,4] = 1$

    y[1] = 1.0; y[2] = 0.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = -0.0; x[4,2] = 0.0; x[4,3] = 1.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.6; x[6,4] = 0.4; 

$Objectif = 1810.0$

$y[2] = 0; x[1,1] = 1, x[4,4] = 0; y[4] = 1; y[3] = 1; x[2,4] = 1; x[5,4] = 1; x[6,4] = 0$

    y[1] = 1.0; y[2] = 0.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = -0.0; x[4,2] = 0.0; x[4,3] = 1.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = -0.0; x[6,2] = 0.0; x[6,3] = 1.0; x[6,4] = 0.0;

$Objectif = 1850.0$

$y[2] = 0; x[1,1] = 1, x[4,4] = 0; y[4] = 1; y[3] = 1; x[2,4] = 1; x[5,4] = 1; x[6,4] = 1$

$Objectif = Infini$

$y[2] = 0; x[1,1] = 1, x[4,4] = 1; y[3] = 0$

$Objectif = Infini$

$y[2] = 0; x[1,1] = 1, x[4,4] = 1; y[3] = 1$

    y[1] = 1.0; y[2] = 0.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 1.0; x[2,4] = 0.0; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 0.0; x[4,3] = 0.0; x[4,4] = 1.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.5714285714285715; x[5,4] = 0.4285714285714285; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 2032.142857142857 > 1850$; on élague

$y[2] = 1; x[1,2] = 0$

    y[1] = 0.8421052631578947; y[2] = 1.0; y[3] = 0.1578947368421053; y[4] = 0.8421052631578947; 
    x[1,1] = 0.8421052631578947; x[1,2] = 0.0; x[1,3] = 0.1578947368421053; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.1578947368421053; x[2,4] = 0.8421052631578947; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.1578947368421053; x[5,4] = 0.8421052631578947; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.1578947368421053; x[6,4] = 0.8421052631578947; 

$Objectif = 1556.578947368421$

$y[2] = 1; x[1,2] = 1$

    y[1] = 0.41353383458646614; y[2] = 1.0; y[3] = 0.1578947368421053; y[4] = 0.8421052631578948; 
    x[1,1] = 0.0; x[1,2] = 1.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.1578947368421052; x[2,4] = 0.8421052631578948; 
    x[3,1] = 0.41353383458646614; x[3,2] = 0.42857142857142855; x[3,3] = 0.1578947368421053; x[3,4] = 0.0; 
    x[4,1] = 0.41353383458646614; x[4,2] = 0.42857142857142855; x[4,3] = 0.1578947368421053; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.1578947368421052; x[5,4] = 0.8421052631578948; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.1578947368421053; x[6,4] = 0.8421052631578947; 

$Objectif = 1647.1804511278197$

$y[2] = 1; x[1,2] = 0; y[1] = 0$

    y[1] = 0.0; y[2] = 1.0; y[3] = 0.44827586206896564; y[4] = 0.5517241379310345; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 0.44827586206896564; x[1,4] = 0.5517241379310344; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.4482758620689655; x[2,4] = 0.5517241379310345; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.4482758620689655; x[5,4] = 0.5517241379310345; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.4482758620689655; x[6,4] = 0.5517241379310345; 

$Objectif = 1642.241379310345$

$y[2] = 1; x[1,2] = 0; y[1] = 1$

    y[1] = 1.0; y[2] = 1.0; y[3] = 0.1578947368421053; y[4] = 0.8421052631578947; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.1578947368421053; x[2,4] = 0.8421052631578947; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.1578947368421053; x[5,4] = 0.8421052631578947; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.1578947368421053; x[6,4] = 0.8421052631578947; 

$Objectif = 1564.4736842105265$

$y[2] = 1; x[1,2] = 0; y[1] = 0; y[4] = 0$

$Objectif = Infini$

$y[2] = 1; x[1,2] = 0; y[1] = 0; y[4] = 1$

    y[1] = 0.0; y[2] = 1.0; y[3] = 0.5416666666666667; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 0.5416666666666667; x[1,4] = 0.45833333333333326; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.5416666666666667; x[2,4] = 0.45833333333333326; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.5416666666666667; x[5,4] = 0.45833333333333326; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 1791.6666666666667$

$y[2] = 1; x[1,2] = 0; y[1] = 0; y[4] = 1; y[3] = 0$

$Objectif = Infini$

$y[2] = 1; x[1,2] = 0; y[1] = 0; y[4] = 1; y[3] = 1$

    y[1] = 0.0; y[2] = 1.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 1.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.4285714285714286; x[5,4] = 0.5714285714285714; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 1817.857142857143$

$y[2] = 1; x[1,2] = 0; y[1] = 0; y[4] = 1; y[3] = 1; x[5,4] = 0$

    y[1] = 0.0; y[2] = 1.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 0.9; x[1,4] = 0.09999999999999998; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = -0.0; x[5,3] = 1.0; x[5,4] = 0.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 1885.0 > 1850$, on élague

$y[2] = 1; x[1,2] = 0; y[1] = 0; y[4] = 1; y[3] = 1; x[5,4] = 1$

    y[1] = 0.0; y[2] = 1.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 1.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.4285714285714286; x[2,4] = 0.5714285714285714; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 1817.857142857143$

$y[2] = 1; x[1,2] = 0; y[1] = 0; y[4] = 1; y[3] = 1; x[5,4] = 1; x[2,4] = 0$

    y[1] = 0.0; y[2] = 1.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 0.9; x[1,4] = 0.09999999999999998; 
    x[2,1] = 0.0; x[2,2] = -0.0; x[2,3] = 1.0; x[2,4] = 0.0; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 1885.0 > 1850$; on élague.

$y[2] = 1; x[1,2] = 0; y[1] = 0; y[4] = 1; y[3] = 1; x[5,4] = 1; x[2,4] = 1$

    y[1] = 0.0; y[2] = 1.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 1.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.6000000000000001; x[6,4] = 0.3999999999999999; 

$Objectif = 1835.0$

$y[2] = 1; x[1,2] = 0; y[1] = 0; y[4] = 1; y[3] = 1; x[5,4] = 1; x[2,4] = 1; x[6,4] = 0$

    y[1] = 0.0; y[2] = 1.0; y[3] = 1.0; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 1.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.0; x[6,2] = -0.0; x[6,3] = 1.0; x[6,4] = 0.0; 

$Objectif = 1875.0$

$y[2] = 1; x[1,2] = 0; y[1] = 0; y[4] = 1; y[3] = 1; x[5,4] = 1; x[2,4] = 1; x[6,4] = 1$

$Objectif = Infini$

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 0$

    y[1] = 1.0; y[2] = 1.0; y[3] = 0.8421052631578947; y[4] = 0.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.1578947368421053; x[2,3] = 0.8421052631578947; x[2,4] = 0.0; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.142857142857143; x[5,2] = 0.015037593984962325; x[5,3] = 0.8421052631578947; x[5,4] = 0.0; 
    x[6,1] = 0.0; x[6,2] = 0.1578947368421053; x[6,3] = 0.8421052631578947; x[6,4] = 0.0; 

$Objectif = 1775.0$

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 1$

    y[1] = 1.0; y[2] = 1.0; y[3] = 0.0; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.4285714285714286; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 0.5714285714285714; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 1610.7142857142858$

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 0; y[3] = 0$

$Objectif = Infini$

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 0; y[3] = 1$

    y[1] = 1.0; y[2] = 1.0; y[3] = 1.0; y[4] = 0.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 1.0; x[2,4] = 0.0; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.4285714285714286; x[5,2] = 0.0; x[5,3] = 0.5714285714285714; x[5,4] = 0.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 1.0; x[6,4] = 0.0; 

$Objectif = 1817.857142857143$

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 0; y[3] = 1; x[5,1] = 0$

    y[1] = 1.0; y[2] = 1.0; y[3] = 1.0; y[4] = 0.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 1.0; x[2,4] = 0.0; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.2857142857142857; x[5,3] = 0.7142857142857143; x[5,4] = 0.0; 
    x[6,1] = 0.19999999999999996; x[6,2] = 0.0; x[6,3] = 0.8; x[6,4] = 0.0; 

$Objectif = 1823.5714285714284$

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 0; y[3] = 1; x[5,1] = 1$

    y[1] = 1.0; y[2] = 1.0; y[3] = 1.0; y[4] = 0.0; 
    x[1,1] = 0.9; x[1,2] = 0.0; x[1,3] = 0.09999999999999998; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 1.0; x[2,4] = 0.0; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 1.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 0.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 1.0; x[6,4] = 0.0; 

$Objectif = 1890.0 > 1850$; on élague.

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 0; y[3] = 1; x[5,1] = 0; x[5,3] = 0$

    y[1] = 1.0; y[2] = 1.0; y[3] = 1.0; y[4] = 0.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 1.0; x[2,4] = 0.0; 
    x[3,1] = 0.8333333333333334; x[3,2] = 0.16666666666666666; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 1.0; x[5,3] = 0.0; x[5,4] = 0.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 1.0; x[6,4] = 0.0; 

$Objectif = 1916.6666666666665 > 1850$; on élague

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 0; y[3] = 1; x[5,1] = 0; x[5,3] = 1$

    y[1] = 1.0; y[2] = 1.0; y[3] = 1.0; y[4] = 0.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.2857142857142857; x[2,3] = 0.7142857142857143; x[2,4] = 0.0; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 1.0; x[5,4] = 0.0; 
    x[6,1] = 0.20000000000000018; x[6,2] = 0.0; x[6,3] = 0.7999999999999998; x[6,4] = 0.0; 

$Objectif = 1823.5714285714287$

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 0; y[3] = 1; x[5,1] = 0; x[5,3] = 1; x[2,3] = 0$

    y[1] = 1.0; y[2] = 1.0; y[3] = 1.0; y[4] = 0.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.7142857142857142; x[2,2] = 0.2857142857142858; x[2,3] = 0.0; x[2,4] = 0.0; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 1.0; x[5,4] = 0.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 1.0; x[6,4] = 0.0; 

$Objectif = 1910.7142857142858  > 1850$; on élague

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 0; y[3] = 1; x[5,1] = 0; x[5,3] = 1; x[2,3] = 1$

    y[1] = 1.0; y[2] = 1.0; y[3] = 1.0; y[4] = 0.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 1.0; x[2,4] = 0.0; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 1.0; x[5,4] = 0.0; 
    x[6,1] = 0.6000000000000001; x[6,2] = 0.0; x[6,3] = 0.3999999999999999; x[6,4] = 0.0; 

$Objectif = 1835.0$

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 0; y[3] = 1; x[5,1] = 0; x[5,3] = 1; x[2,3] = 1; x[6,3] = 0$

    y[1] = 1.0; y[2] = 1.0; y[3] = 1.0; y[4] = 0.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 1.0; x[2,4] = 0.0; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 1.0; x[5,4] = 0.0; 
    x[6,1] = 0.6000000000000001; x[6,2] = 0.3999999999999999; x[6,3] = 0.0; x[6,4] = 0.0; 

$Objectif = 1875.0 > 1850$; on élage

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 0; y[3] = 1; x[5,1] = 0; x[5,3] = 1; x[2,3] = 1; x[6,3] = 1$

$Objectif = Infini$

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 1; x[5,4] = 0$

    y[1] = 1.0; y[2] = 1.0; y[3] = 0.0; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.7142857142857143; x[5,2] = 0.2857142857142857; x[5,3] = 0.0; x[5,4] = 0.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 1725.0$

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 1; x[5,4] = 1$

    y[1] = 1.0; y[2] = 1.0; y[3] = 0.0; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.1428571428571429; x[2,2] = 0.2857142857142857; x[2,3] = 0.0; x[2,4] = 0.5714285714285714; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 1617.857142857143$

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 1; x[5,4] = 0; x[5,1] = 0$

    y[1] = 1.0; y[2] = 1.0; y[3] = 0.0; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 0.8333333333333334; x[3,2] = 0.16666666666666666; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 1.0; x[5,3] = 0.0; x[5,4] = 0.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 1766.6666666666667$


$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 1; x[5,4] = 0; x[5,1] = 1$

    y[1] = 1.0; y[2] = 1.0; y[3] = 0.0; y[4] = 1.0; 
    x[1,1] = 0.9; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.09999999999999998; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 1.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 0.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 1750.0$

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 1; x[5,4] = 0; x[5,1] = 0; x[3,1] = 0$

    y[1] = 1.0; y[2] = 1.0; y[3] = 0.0; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.625; x[4,2] = 0.375; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 1.0; x[5,3] = 0.0; x[5,4] = 0.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 1803.125$

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 1; x[5,4] = 0; x[5,1] = 0; x[3,1] = 1$

    y[1] = 1.0; y[2] = 1.0; y[3] = 0.0; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 1.0; x[5,3] = 0.0; x[5,4] = 0.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 1775.0$

L'intervalle devient: $$]1464.9122807017545,_ \ 1775]$$

On élague donc le sous arbre $y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 1; x[5,4] = 0; x[5,1] = 0; x[3,1] = 0$ car:

$Objectif = 1803.125 > 1775$

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 1; x[5,4] = 0; x[5,1] = 1; x[1,1] = 0$

    y[1] = 1.0; y[2] = 1.0; y[3] = 0.35294117647058826; y[4] = 1.0; 
    x[1,1] = 0.0; x[1,2] = 0.0; x[1,3] = 0.35294117647058826; x[1,4] = 0.6470588235294117; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.35294117647058826; x[2,4] = 0.6470588235294117; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 1.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 0.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 2080.8823529411766 > 1775$; on élague

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 1; x[5,4] = 0; x[5,1] = 1; x[1,1] = 1$

$Objectif = Infini$

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 1; x[5,4] = 1; x[2,4] = 0$

    y[1] = 1.0; y[2] = 1.0; y[3] = 0.0; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.7142857142857143; x[2,2] = 0.2857142857142857; x[2,3] = 0.0; x[2,4] = 0.0; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 1760.7142857142858$

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 1; x[5,4] = 1; x[2,4] = 1 $

    y[1] = 1.0; y[2] = 1.0; y[3] = 0.0; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.6; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 0.4; 

$Objectif = 1645.0$

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 1; x[5,4] = 1; x[2,4] = 0; x[2,1] = 0$

    y[1] = 1.0; y[2] = 1.0; y[3] = 0.0; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 1.0; x[2,3] = 0.0; x[2,4] = 0.0; 
    x[3,1] = 0.8333333333333334; x[3,2] = 0.16666666666666666; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 1766.6666666666665$

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 1; x[5,4] = 1; x[2,4] = 0; x[2,1] = 1$

    y[1] = 1.0; y[2] = 1.0; y[3] = 0.0; y[4] = 1.0; 
    x[1,1] = 0.9; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.09999999999999998; 
    x[2,1] = 1.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 0.0; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 1800.0 > 1775$; on élague

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 1; x[5,4] = 1; x[2,4] = 0; x[2,1] = 0; x[3,1] = 0$

    y[1] = 1.0; y[2] = 1.0; y[3] = 0.0; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 1.0; x[2,3] = 0.0; x[2,4] = 0.0; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.625; x[4,2] = 0.375; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 1803.125 > 1775$; on élague.

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 1; x[5,4] = 1; x[2,4] = 0; x[2,1] = 0; x[3,1] = 1$

    y[1] = 1.0; y[2] = 1.0; y[3] = 0.0; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 1.0; x[2,3] = 0.0; x[2,4] = 0.0; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 1.0; 

$Objectif = 1775.0$

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 1; x[5,4] = 1; x[2,4] = 1; x[6,1] = 0$

    y[1] = 1.0; y[2] = 1.0; y[3] = 0.0; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 0.16666666666666674; x[3,2] = 0.8333333333333333; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.0; x[6,2] = 0.6; x[6,3] = 0.0; x[6,4] = 0.4; 

$Objectif = 1653.3333333333335$

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 1; x[5,4] = 1; x[2,4] = 1; x[6,1] = 1$

    y[1] = 1.0; y[2] = 1.0; y[3] = 0.0; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 1.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 0.0; 

$Objectif = 1725.0$

L'intervalle devient: $$]1464.9122807017545,_ \ 1725]$$

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 1; x[5,4] = 1; x[2,4] = 1; x[6,1] = 0; x[3,2] = 0$

    y[1] = 1.0; y[2] = 1.0; y[3] = 0.0; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.0; x[6,2] = 0.6; x[6,3] = 0.0; x[6,4] = 0.4; 

$Objectif = 1695.0$

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 1; x[5,4] = 1; x[2,4] = 1; x[6,1] = 0; x[3,2] = 1$

    y[1] = 1.0; y[2] = 1.0; y[3] = 0.0; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.125; x[4,2] = 0.875; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.0; x[6,2] = 0.6; x[6,3] = 0.0; x[6,4] = 0.4; 

$Objectif = 1660.625$

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 1; x[5,4] = 1; x[2,4] = 1; x[6,1] = 0; x[3,2] = 0; x[6,4] = 0$

    y[1] = 1.0; y[2] = 1.0; y[3] = 0.0; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 1.0; x[3,2] = 0.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.0; x[6,2] = 1.0; x[6,3] = 0.0; x[6,4] = 0.0; 

$Objectif = 1775.0$

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 1; x[5,4] = 1; x[2,4] = 1; x[6,1] = 0; x[3,2] = 0; x[6,4] = 1$

$Objectif = Infini$

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 1; x[5,4] = 1; x[2,4] = 1; x[6,1] = 0; x[3,2] = 1; x[6,2] = 0$

    y[1] = 1.0; y[2] = 1.0; y[3] = 0.6; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.6; x[6,4] = 0.4; 

$Objectif = 1765.0 > 1725$; on élague

$y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 1; x[5,4] = 1; x[2,4] = 1; x[6,1] = 0; x[3,2] = 1; x[6,2] = 1$

    y[1] = 1.0; y[2] = 1.0; y[3] = 0.0; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.375; x[4,2] = 0.625; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 0.0; x[6,2] = 1.0; x[6,3] = 0.0; x[6,4] = 0.0; 

$Objectif = 1771.875 > 1725$; on élague

$y[2] = 1; x[1,2] = 1; x[3,2] = 0$

    y[1] = 0.0; y[2] = 1.0; y[3] = 0.36; y[4] = 0.6399999999999999; 
    x[1,1] = 0.0; x[1,2] = 1.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.3600000000000001; x[2,4] = 0.6399999999999999; 
    x[3,1] = 0.0; x[3,2] = 0.0; x[3,3] = 0.36; x[3,4] = 0.64; 
    x[4,1] = 0.0; x[4,2] = 0.75; x[4,3] = 0.25; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.36; x[5,4] = 0.64; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.36; x[6,4] = 0.64; 

$Objectif = 1708.25$

$y[2] = 1; x[1,2] = 1; x[3,2] = 1$

    y[1] = 0.0; y[2] = 1.0; y[3] = 0.40740740740740744; y[4] = 0.5925925925925927; 
    x[1,1] = 0.0; x[1,2] = 1.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.40740740740740744; x[2,4] = 0.5925925925925926; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 0.0; x[4,3] = 0.40740740740740744; x[4,4] = 0.5925925925925926; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.40740740740740733; x[5,4] = 0.5925925925925927; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.40740740740740733; x[6,4] = 0.5925925925925927; 

$Objectif = 1725.9259259259259 > 1725$; on élague

$y[2] = 1; x[1,2] = 1; x[3,2] = 0; x[2,3] = 0$

    y[1] = 0.8421052631578947; y[2] = 1.0; y[3] = 0.1578947368421053; y[4] = 0.8421052631578947; 
    x[1,1] = 0.0; x[1,2] = 1.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.1578947368421053; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 0.8421052631578947; 
    x[3,1] = 0.8421052631578947; x[3,2] = 0.0; x[3,3] = 0.1578947368421053; x[3,4] = 0.0; 
    x[4,1] = 0.25; x[4,2] = 0.75; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.1578947368421053; x[5,4] = 0.8421052631578948; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.1578947368421053; x[6,4] = 0.8421052631578947; 

$Objectif = 1737.828947368421 > 1725$; on élague

$y[2] = 1; x[1,2] = 1; x[3,2] = 0; x[2,3] = 1$

    y[1] = 0.0; y[2] = 1.0; y[3] = 1.0; y[4] = 0.9166666666666667; 
    x[1,1] = 0.0; x[1,2] = 1.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 1.0; x[2,4] = 0.0; 
    x[3,1] = 0.0; x[3,2] = 0.0; x[3,3] = 1.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 0.75; x[4,3] = 0.25; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.08333333333333334; x[5,4] = 0.9166666666666667; 
    x[6,1] = 0.0; x[6,2] = 0.0; x[6,3] = 0.08333333333333333; x[6,4] = 0.9166666666666667; 

$Objectif = 1893.75 > 1725$; on élague

### Solution optimale

De tout ce qui précède, on obtient bien une solution optimale déduite du sous arbre $y[2] = 1; x[1,2] = 0; y[1] = 1; y[4] = 1; x[5,4] = 1; x[2,4] = 1; x[6,1] = 1$ donnant comme variables: 

    y[1] = 1.0; y[2] = 1.0; y[3] = 0.0; y[4] = 1.0; 
    x[1,1] = 1.0; x[1,2] = 0.0; x[1,3] = 0.0; x[1,4] = 0.0; 
    x[2,1] = 0.0; x[2,2] = 0.0; x[2,3] = 0.0; x[2,4] = 1.0; 
    x[3,1] = 0.0; x[3,2] = 1.0; x[3,3] = 0.0; x[3,4] = 0.0; 
    x[4,1] = 0.0; x[4,2] = 1.0; x[4,3] = 0.0; x[4,4] = 0.0; 
    x[5,1] = 0.0; x[5,2] = 0.0; x[5,3] = 0.0; x[5,4] = 1.0; 
    x[6,1] = 1.0; x[6,2] = 0.0; x[6,3] = 0.0; x[6,4] = 0.0; 

$Objectif = 1725.0$

# Conclusion

Un entrepôt doit être ouvert dans chacunes des villes suivantes:   
- Brill
- Hearthglen 
- Strahnbrad

Les affectations doivent être faites comme suit:

- La capitale Voldemort --> Brill
- La citadelle de Dalaran --> Strahnbrad
- La cité de Stratholme --> Hearthglen
- La forteresse de Mardenhol --> Hearthglen
- La forteresse de Durnholde --> Strahnbrad
- La cité de Caer --> Brill